In [ ]:
#import os
# make sure pyspark tells workers to use python3 not 2 if both are installed
#os.environ['PYSPARK_PYTHON'] = '/home/ubuntu/anaconda3/bin/python3'
#os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/ubuntu/anaconda3/bin/ipython'
#!pip install metapy

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, expr, when, concat, lit, isnan
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.feature import VectorIndexer, VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.ml import Pipeline
import pyspark
from pyspark.sql import SQLContext
import metapy

In [2]:
#Al ejecutar esta celda se demora un poco, así que un poco de paciencia
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local', "news_cleaned_topic_model") 
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [3]:
dfNews=spark.read.csv("file:///home/ubuntu/Caso_estudio/datasets/news.csv", inferSchema=True, header=True, encoding="UTF-8")
dfNews.show(10)


+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
| id|id_news|               title|   publication|              author|      date|  year|month| url|             content|
+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|  0|  17283|House Republicans...|New York Times|          Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON  —   C...|
|  1|  17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0|  6.0|null|After the bullet ...|
|  2|  17285|Tyrus Wong, ‘Bamb...|New York Times|        Margalit Fox|2017-01-06|2017.0|  1.0|null|When Walt Disney’...|
|  3|  17286|Among Deaths in 2...|New York Times|    William McDonald|2017-04-10|2017.0|  4.0|null|Death may be the ...|
|  4|  17287|Kim Jong-un Says ...|New York Times|       Choe Sang-Hun|2017-01-02|2017.0|  1.0|null|SEOUL, South Kore...|
|  5|  17288|Sick With a Cold,..

In [4]:
dfNews.count()

185186

In [5]:
dfNewsFiltered=dfNews.na.drop(subset=("title","publication","author","year", "date", "month","content"))

In [6]:
dfNewsFiltered

DataFrame[id: string, id_news: string, title: string, publication: string, author: string, date: string, year: string, month: string, url: string, content: string]

In [7]:
dfNewsFiltered.count()

129519

In [8]:
import pyspark.sql.functions as F

dfStatistics =dfNewsFiltered.groupBy("publication").agg(F.countDistinct("publication"))
#dfNewsFiltered.groupBy("publication").count().orderBy().show()

#dfNewsFiltered.registerTempTable("TblNews")
#dfStatistics = sqlContext.sql("select publication, year, count(distinct publication) from TblNews groupby publication")
#dfStatistics.show()



In [ ]:
#dfStatistics.sort(desc("count(DISTINCT publication)")

#!rm -rf news-idx
#idx = metapy.index.make_inverted_index('/home/ubuntu/bin/meta/build/config_news.toml')

#print(f'Número Total de noticias: {idx.num_docs()}')
#print(f'Número de palabras únicas: {idx.unique_terms()}')
#print(f'Promedio de longitud de las documentos: {idx.avg_doc_length()}')

In [9]:
dfMainCols= dfNewsFiltered.select('id_news','title','publication','content')
dfMainCols.show(10)

+-------+--------------------+--------------+--------------------+
|id_news|               title|   publication|             content|
+-------+--------------------+--------------+--------------------+
|  17283|House Republicans...|New York Times|WASHINGTON  —   C...|
|  17284|Rift Between Offi...|New York Times|After the bullet ...|
|  17285|Tyrus Wong, ‘Bamb...|New York Times|When Walt Disney’...|
|  17286|Among Deaths in 2...|New York Times|Death may be the ...|
|  17287|Kim Jong-un Says ...|New York Times|SEOUL, South Kore...|
|  17288|Sick With a Cold,...|New York Times|LONDON  —   Queen...|
|  17289|Taiwan’s Presiden...|New York Times|BEIJING  —   Pres...|
|  17290|After ‘The Bigges...|New York Times|Danny Cahill stoo...|
|  17291|First, a Mixtape....|New York Times|Just how   is Hil...|
|  17292|Calling on Angels...|New York Times|Angels are everyw...|
+-------+--------------------+--------------+--------------------+
only showing top 10 rows



In [10]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
#from pyspark.sql.functions import monotonically_increasing_id
import re

In [11]:
import nltk
import pandas as pd
import numpy as np
import codecs
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer

In [ ]:
#publishers= dfMainCols.select("publication").distinct()
#publishers.show(50)

In [12]:
# Default list of Stopwords
stopwords_core = ['a', u'about', u'above', u'after', u'again', u'against', u'all', u'am', u'an', u'and', u'any', 
                  u'are', u'arent', u'as', u'at', u'be', u'because', u'been', u'before', u'being', u'below', 
                  u'between', u'both', u'but', u'by',     u'can', 'cant', 'come', u'could', 'couldnt',u'd', u'did', 
                  u'didn', u'do', u'does', u'doesnt', u'doing', u'dont', u'down', u'during', u'each', u'few', u'finally', 
                  u'for', u'from', u'further', u'had', u'hadnt', u'has', u'hasnt', u'have', u'havent', u'having', u'he', 
                  u'her', u'here', u'hers', u'herself', u'him', u'himself', u'his', u'how', u'i', u'if', u'in', u'into', 
                  u'is', u'isnt', u'it', u'its', u'itself', u'just', u'll',u'm', u'me', u'might', u'more', u'most', u'must',
                  u'my', u'myself', u'no', u'nor', u'not', u'now', u'o', u'of', u'off', u'on', u'once', u'only', u'or', 
                  u'other', u'our', u'ours', u'ourselves', u'out', u'over', u'own', u'r', u're', u's', 'said', u'same', 
                  u'she', u'should', u'shouldnt', u'so', u'some', u'such', u't', u'than', u'that', 'thats', u'the', u'their',
                  u'theirs', u'them', u'themselves', u'then', u'there', u'these', u'they', u'this', u'those', u'through', 
                  u'to', u'too', u'under', u'until', u'up',u'very', u'was', u'wasnt', u'we', u'were', u'werent', u'what', 
                  u'when', u'where', u'which', u'while', u'who', u'whom', u'why', u'will', u'with', u'wont', u'would', u'y', 
                  u'you', u'your', u'yours', u'yourself', u'yourselves']
    
# Custom List of Stopwords - Add your own here
stopwords_custom = ['New', 'York','Times','Breitbart', 'News']
stopwords = stopwords_core + stopwords_custom
stopwords = [word.lower() for word in stopwords] 

#Stemmer
ps = PorterStemmer() 

def cleanup_text(record):
    textTitle  = str(record[1])
    textPublication=str(record[2])
    textContent = str(record[3])
    
    #setStopwordsCustom =set(stopwords_custom)
    #setStopwordsCustom =setStopwordsCustom.union(set(textPublication.split()))
    #stopwords_custom1 = list(setStopwordsCustom)
    #stopwords_custom= stopwords_custom1
    #stopwords = stopwords_core + stopwords_custom1
    #title=textTitle
    #longPublication =len(textPublication) # Calcular la longitud de la cadena de caracteres  del publisher de la noticia
    #longTitle=len(textTitle)  # Calcular la longitud de la cadena del titulo de la noticia
    
    #if longTitle > longPublication:
    #    textPublisher=textTitle[longTitle-longPublication-1:longTitle] # Extrar caracteres derechos del titulo
    #    if textPublisher==textPublication:
    #        title=textTitle[0:longTitle-longPublication-1]
    #    else:
    #        title=textTitle
    
    textCombined = textTitle + " " + textContent
    # Remove special characters
    textCombined= re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)\n','',textCombined)
    textCombined= re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)',' ',textCombined)
    textCombined= re.sub('[^(a-zA-Z)]',' ',textCombined)
    textCombined= re.sub(' +',' ',textCombined)
    words = textCombined.split()
    tokens3 = [word.lower() for word in words if len(word)>3 and word.lower() not in stopwords] 
    text_out = [ps.stem(w) for w in tokens3]
    
    #text_out = [re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)\n','',word) for word in words]
    #text_out = [re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)',' ',word) for word in words]
    #text_out = [re.sub('[^(a-zA-Z)]',' ',word) for word in words]                                   
    #text_out = [re.sub(' +',' ',word) for word in words] 
    # Remove stopwords and words under X length
    return text_out

In [13]:
udf_cleantext = udf(cleanup_text , ArrayType(StringType()))
dfCombined = dfMainCols.withColumn("CombinedTokens", udf_cleantext(struct([dfMainCols[x] for x in dfMainCols.columns])))

In [14]:
dfCombined.show(10)

+-------+--------------------+--------------+--------------------+--------------------+
|id_news|               title|   publication|             content|      CombinedTokens|
+-------+--------------------+--------------+--------------------+--------------------+
|  17283|House Republicans...|New York Times|WASHINGTON  —   C...|[hous, republican...|
|  17284|Rift Between Offi...|New York Times|After the bullet ...|[rift, offic, res...|
|  17285|Tyrus Wong, ‘Bamb...|New York Times|When Walt Disney’...|[tyru, wong, bamb...|
|  17286|Among Deaths in 2...|New York Times|Death may be the ...|[among, death, he...|
|  17287|Kim Jong-un Says ...|New York Times|SEOUL, South Kore...|[jong, say, north...|
|  17288|Sick With a Cold,...|New York Times|LONDON  —   Queen...|[sick, cold, quee...|
|  17289|Taiwan’s Presiden...|New York Times|BEIJING  —   Pres...|[taiwan, presid, ...|
|  17290|After ‘The Bigges...|New York Times|Danny Cahill stoo...|[biggest, loser, ...|
|  17291|First, a Mixtape....|Ne

In [15]:
# Term Frequency Vectorization: 
cvTokensCombined = CountVectorizer(inputCol="CombinedTokens", outputCol="rawFeatures", vocabSize = 500)
cvmodel = cvTokensCombined.fit(dfCombined)
featurizedData = cvmodel.transform(dfCombined)

vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)

#Calculating IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

KeyboardInterrupt: 

In [ ]:
rescaledData.show(10)

In [ ]:
# Generate 20 Data-Driven Topics:
lda = LDA(k=20, seed=123, optimizer="em", featuresCol="features")

ldamodel = lda.fit(rescaledData)

#model.isDistributed()
#model.vocabSize()

ldatopics = ldamodel.describeTopics()
ldatopics.show(3)

In [ ]:
featurizedData.take(10)

In [ ]:
def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])
    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))
ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(20,False)

In [ ]:
ldaResults = ldamodel.transform(rescaledData)

ldaResults.select('id_news','title','publication','content','CombinedTokens','features','topicDistribution').show()

In [ ]:
ldaResults.select('id_news','topicDistribution').show()

In [ ]:
def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])
    
    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))
ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(20,False)

In [ ]:
ldaResults.topicDistribution

In [ ]:
from pyspark.sql.types import IntegerType


def asignTopic(topicDistribution):
    topic = topicDistribution[0]
    index_topic = 0
    for index in range(len(topicDistribution)):
        if (topicDistribution[index]>topic):
            topic=topicDistribution[index]
            index_topic=index
    return index_topic

udf_asignTopic = udf(asignTopic, IntegerType())
dfMainTopic = ldaResults.withColumn("mainTopic", udf_asignTopic(ldaResults.topicDistribution))

In [ ]:
dfNewsTopic.select('mainTopic').show()

# Sentimientos

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
from pyspark.sql.types import StructType,ArrayType,StringType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer

In [20]:
dat=pd.read_csv("file:///home/ubuntu/Caso_estudio/datasets/mini_news.csv")

### Limpieza de datos
Se elimina del titulo la publicación, esto ayuda a mejorar la clasifiación de los sentimientos de la noticia

In [21]:
#dat=dat[dat['title'].fillna(0, inplace=True)]
zipped = zip(dat['title'], dat['content'])
title=[]
content=[]
sep = ' - '
for i,j in zipped:
    if pd.notna(i):
        tit=i
        p = tit.split(sep, 1)[0]
        content.append(j)
        title.append(p)
    else:
        title.append(0)

### Identificación del sentimiento
0: negativo,
1: neutro,
2: positivo

In [22]:
target=[]
sid = SentimentIntensityAnalyzer()
for t in title:
    comp=sid.polarity_scores(t)['compound'] # Score de sentimiento
    #print(comp)
    if comp<0:
        lb=0
    elif comp==0:
        lb=1
    else:
        lb=2
    target.append(lb)

In [23]:
d = {'title':title,'content':content,'target':target}
dfsent = pd.DataFrame(d)

In [24]:
dfsent.head(5)

,title,content,target
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...,2
1,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ...",1
2,"Sick With a Cold, Queen Elizabeth Misses New Y...","LONDON — Queen Elizabeth II, who has been b...",0
3,Taiwan’s President Accuses China of Renewed In...,BEIJING — President Tsai of Taiwan sharpl...,0
4,Calling on Angels While Enduring the Trials of...,Angels are everywhere in the Muñiz family’s ap...,1


In [25]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
spark_dfsent = sqlContext.createDataFrame(dfsent)

In [26]:
spark_dfsent.show(5)

+--------------------+--------------------+------+
|               title|             content|target|
+--------------------+--------------------+------+
|House Republicans...|WASHINGTON  —   C...|     2|
|Kim Jong-un Says ...|SEOUL, South Kore...|     1|
|Sick With a Cold,...|LONDON  —   Queen...|     0|
|Taiwan’s Presiden...|BEIJING  —   Pres...|     0|
|Calling on Angels...|Angels are everyw...|     1|
+--------------------+--------------------+------+
only showing top 5 rows



In [27]:
df_clean=spark_dfsent.select(lower(regexp_replace(spark_dfsent.content, "[^a-zA-Z\\s]", "")).alias('text'),spark_dfsent.target)
df_clean.show(5)

+--------------------+------+
|                text|target|
+--------------------+------+
|washington     co...|     2|
|seoul south korea...|     1|
|london     queen ...|     0|
|beijing     presi...|     0|
|angels are everyw...|     1|
+--------------------+------+
only showing top 5 rows



In [28]:
tokenizer = Tokenizer(inputCol='text', outputCol='words_token')
df_words_token = tokenizer.transform(df_clean).select('text', 'words_token','target')
df_words_token.show(5)

+--------------------+--------------------+------+
|                text|         words_token|target|
+--------------------+--------------------+------+
|washington     co...|[washington, , , ...|     2|
|seoul south korea...|[seoul, south, ko...|     1|
|london     queen ...|[london, , , , , ...|     0|
|beijing     presi...|[beijing, , , , ,...|     0|
|angels are everyw...|[angels, are, eve...|     1|
+--------------------+--------------------+------+
only showing top 5 rows



In [29]:
locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
df_words_no_stopw = remover.transform(df_words_token).select('words_token', 'words_clean','target')
df_words_no_stopw.show(5)

+--------------------+--------------------+------+
|         words_token|         words_clean|target|
+--------------------+--------------------+------+
|[washington, , , ...|[washington, , , ...|     2|
|[seoul, south, ko...|[seoul, south, ko...|     1|
|[london, , , , , ...|[london, , , , , ...|     0|
|[beijing, , , , ,...|[beijing, , , , ,...|     0|
|[angels, are, eve...|[angels, everywhe...|     1|
+--------------------+--------------------+------+
only showing top 5 rows



# Modelos

### Datos

In [30]:
train, test = df_clean.randomSplit([0.7, 0.3], seed = 2019)
print("Training Dataset: " + str(train.count()))
print("Test Dataset: " + str(test.count()))

Training Dataset: 168
Test Dataset: 81


### Preparacióm de datos co Pipeline

In [32]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])

pipelineFit = pipeline.fit(train)
train_df = pipelineFit.transform(train)
val_df = pipelineFit.transform(test)

In [33]:
train_df.show(10)

+--------------------+------+--------------------+--------------------+--------------------+-----+
|                text|target|               words|                  tf|            features|label|
+--------------------+------+--------------------+--------------------+--------------------+-----+
|  donald j trump ...|     0|[, , donald, j, t...|(65536,[65,88,556...|(65536,[65,88,556...|  1.0|
|  donald j trump ...|     0|[, , donald, j, t...|(65536,[187,389,5...|(65536,[187,389,5...|  1.0|
| want to get this...|     2|[, want, to, get,...|(65536,[12,110,12...|(65536,[12,110,12...|  2.0|
|a controversial e...|     2|[a, controversial...|(65536,[61,338,49...|(65536,[61,338,49...|  2.0|
|a few hours after...|     1|[a, few, hours, a...|(65536,[14,150,55...|(65536,[14,150,55...|  0.0|
|a florida man has...|     0|[a, florida, man,...|(65536,[308,338,5...|(65536,[308,338,5...|  1.0|
|a long island rai...|     1|[a, long, island,...|(65536,[209,368,3...|(65536,[209,368,3...|  0.0|
|a man who

## RadomForest

In [34]:
from pyspark.ml.classification import RandomForestClassifier

pipelineFit = pipeline.fit(train)
train_df = pipelineFit.transform(train)
test_df = pipelineFit.transform(test)

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train_df)
predictions_Forest = rfModel.transform(test_df)


In [35]:
predictions_Forest.select('label', 'rawPrediction', 'prediction', 'probability').show(20)

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|  0.0|[11.0932481879091...|       0.0|[0.55466240939545...|
|  0.0|[8.97017324623525...|       0.0|[0.44850866231176...|
|  0.0|[10.4919579134395...|       0.0|[0.52459789567197...|
|  1.0|[9.79037011562038...|       0.0|[0.48951850578101...|
|  2.0|[9.22143251870938...|       0.0|[0.46107162593546...|
|  1.0|[11.4882028182988...|       0.0|[0.57441014091494...|
|  1.0|[10.6830839819691...|       0.0|[0.53415419909845...|
|  2.0|[10.2451587222944...|       0.0|[0.51225793611472...|
|  1.0|[9.89556686359638...|       0.0|[0.49477834317981...|
|  1.0|[8.82958527230802...|       0.0|[0.44147926361540...|
|  1.0|[10.7393600937554...|       0.0|[0.53696800468777...|
|  1.0|[9.05098050683347...|       0.0|[0.45254902534167...|
|  1.0|[11.3812948512002...|       0.0|[0.56906474256001...|
|  0.0|[9.08637573344374

In [36]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions_Forest, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.5499999999999999


### K-folds

In [37]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())


In [38]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(train_df)

In [39]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(test_df)

In [40]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.4925925925925926

In [ ]:
# View Best model's predictions and probabilities of each prediction class
#selected = predictions.select("label", "prediction", "probability", "age", "occupation")
#display(selected)

## Multinomial

In [41]:
from pyspark.ml.classification import LogisticRegression
mlr = LogisticRegression(featuresCol = 'features', labelCol = 'label',maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")
mlrModel = mlr.fit(train_df)
mlpredictions = mlrModel.transform(test_df)
mlpredictions.select('target','label', 'rawPrediction', 'prediction', 'probability').show(10)


+------+-----+--------------------+----------+--------------------+
|target|label|       rawPrediction|prediction|         probability|
+------+-----+--------------------+----------+--------------------+
|     1|  0.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     1|  0.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     1|  0.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     0|  1.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     2|  2.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     0|  1.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     0|  1.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     2|  2.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     0|  1.0|[0.45371872668574...|       0.0|[0.48214244232133...|
|     0|  1.0|[0.45371872668574...|       0.0|[0.48214244232133...|
+------+-----+--------------------+----------+--------------------+
only showing top 10 rows



In [42]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(mlpredictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.5
